## How to query a Nexus SPARQL View ?

This tutorial demonstrates how to:
* run a query against an existing Nexus Sparql View
* convert the sparql result to a pandas data frame


In [ ]:
#Let install sparqlwrapper which a python wrapper around sparql client
!pip install git+https://github.com/RDFLib/sparqlwrapper

In [ ]:
import pandas as pd
from itertools import repeat
import concurrent.futures
import json
import csv
import requests
import time
from urllib.parse import urlencode, quote_plus
from collections import OrderedDict
import pandas as pd
from io import StringIO
from functools import reduce
from SPARQLWrapper import SPARQLWrapper, XML, N3, TURTLE, JSON, POST, POSTDIRECTLY, CSV

In [ ]:
#Configure a sparql client for a Nexus Sparql View

token ="Put Your Token Here"
ns = "$NEXUS-URL"
endpoint = ns+"views/tutorialnexus/$PROJECTLABEL/graph/sparql/"
headers = {}
headers["Authorization"] ="Bearer {}".format(token)
headers["Content-Type"] ="application/sparql-query"

sparql_client = SPARQLWrapper(endpoint)
sparql_client.addCustomHttpHeader("Content-Type", "application/sparql-query")
sparql_client.addCustomHttpHeader("Authorization","Bearer {}".format(token))
sparql_client.setMethod(POST)
sparql_client.setReturnFormat(JSON)
sparql_client.setRequestMethod(POSTDIRECTLY)

In [ ]:
# Convert SPARQL results into a Pandas data frame
def sparql2dataframe(json_sparql_results):
    cols = json_sparql_results['head']['vars']
    out = []
    for row in json_sparql_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)
    return pd.DataFrame(out, columns=cols)

#Use a client and send a query
def query(query, sparql_client):
    sparql_client.setQuery(query)
    result_object = sparql_client.query()
    return result_object._convertJSON()
   
    

In [ ]:
# Configure and run query

querystring = """
PREFIX vocab: <$NEXUS-URL/vocabs/tutorialnexus/$PROJECTLABEL/>
PREFIX nxv: <https://bluebrain.github.io/nexus/vocabulary/>

Select ?userId ?movieId  ?rating
 WHERE  {
    ?ratingNode a vocab:Rating.
    ?ratingNode vocab:movieId ?movieId.
    ?ratingNode vocab:rating ?rating.
    ?ratingNode vocab:userId ?userId.

}
ORDER BY ASC(?userId)
LIMIT 250"""



results = query(querystring,sparql_client)

# Convert to a dataframe

df = sparql2dataframe(results)
df.head()





